![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/8.Generic_Classifier.ipynb)

# 8. Generic Classifier

**NOTE**: 

All the components in this notebook work properly in these versions:

*   sparknlp: 3.4.2
*   sparknlp_jsl: 3.4.2
*   pyspark: 3.1.2

Please keep in mind you may have some issues if you use any other version.

In [ ]:
import json
import os
from google.colab import files

license_keys = files.upload()

with open(list(license_keys.keys())[0]) as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)

# Adding license key-value pairs to environment variables
os.environ.update(license_keys)

In [ ]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.1.2 spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

In [3]:
import json
import os

from pyspark.ml import Pipeline
from pyspark.sql import SparkSession

import sparknlp
import sparknlp_jsl

from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *

params = {"spark.driver.memory":"16G",
          "spark.kryoserializer.buffer.max":"2000M",
          "spark.driver.maxResultSize":"2000M"}

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

print("Spark NLP Version :", sparknlp.version())
print("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark

Spark NLP Version : 3.4.2
Spark NLP_JSL Version : 3.4.2


In [4]:
 # if you want to start the session with custom params as in start function above

def start(secret):
    builder = SparkSession.builder \
        .appName("Spark NLP Licensed") \
        .master("local[*]") \
        .config("spark.driver.memory", "16G") \
        .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
        .config("spark.kryoserializer.buffer.max", "2000M") \
        .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.11:"+version) \
        .config("spark.jars", "https://pypi.johnsnowlabs.com/"+secret+"/spark-nlp-jsl-"+jsl_version+".jar")
      
    return builder.getOrCreate()


#spark = start(secret)

## load dataset

In [5]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/petfinder-mini.csv

In [6]:
import pandas as pd
dataframe = pd.read_csv('petfinder-mini.csv')

In [7]:
# In the original dataset "4" indicates the pet was not adopted.
import numpy as np
dataframe['target'] = np.where(dataframe['AdoptionSpeed']==4, 0, 1)

In [8]:
dataframe = dataframe.drop(['AdoptionSpeed'], axis=1)

In [9]:
dataframe.head()

,Type,Age,Breed1,Gender,Color1,Color2,MaturitySize,FurLength,Vaccinated,Sterilized,Health,Fee,Description,PhotoAmt,target
0,Cat,3,Tabby,Male,Black,White,Small,Short,No,No,Healthy,100,Nibble is a 3+ month old ball of cuteness. He ...,1,1
1,Cat,1,Domestic Medium Hair,Male,Black,Brown,Medium,Medium,Not Sure,Not Sure,Healthy,0,I just found it alone yesterday near my apartm...,2,1
2,Dog,1,Mixed Breed,Male,Brown,White,Medium,Medium,Yes,No,Healthy,0,Their pregnant mother was dumped by her irresp...,7,1
3,Dog,4,Mixed Breed,Female,Black,Brown,Medium,Short,Yes,No,Healthy,150,"Good guard dog, very alert, active, obedience ...",8,1
4,Dog,1,Mixed Breed,Male,Black,No Color,Medium,Short,No,No,Healthy,0,This handsome yet cute boy is up for adoption....,3,1


In [10]:
dataframe.columns

Index(['Type', 'Age', 'Breed1', 'Gender', 'Color1', 'Color2', 'MaturitySize',
       'FurLength', 'Vaccinated', 'Sterilized', 'Health', 'Fee', 'Description',
       'PhotoAmt', 'target'],
      dtype='object')

In [11]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11537 entries, 0 to 11536
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Type          11537 non-null  object
 1   Age           11537 non-null  int64 
 2   Breed1        11537 non-null  object
 3   Gender        11537 non-null  object
 4   Color1        11537 non-null  object
 5   Color2        11537 non-null  object
 6   MaturitySize  11537 non-null  object
 7   FurLength     11537 non-null  object
 8   Vaccinated    11537 non-null  object
 9   Sterilized    11537 non-null  object
 10  Health        11537 non-null  object
 11  Fee           11537 non-null  int64 
 12  Description   11528 non-null  object
 13  PhotoAmt      11537 non-null  int64 
 14  target        11537 non-null  int64 
dtypes: int64(4), object(11)
memory usage: 1.3+ MB


In [12]:
dataframe.target.value_counts()

1    8457
0    3080
Name: target, dtype: int64

In [13]:
dataframe.Description = dataframe.Description.fillna('- no description -')

## Featurize with Sklearn Column Transformer

In [14]:
from sklearn.compose import make_column_transformer
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

column_trans = make_column_transformer(
     (OneHotEncoder(), ['Type', 'Breed1', 'Gender', 'Color1', 'Color2', 'MaturitySize',
       'FurLength', 'Vaccinated', 'Sterilized', 'Health']),
     (TfidfVectorizer(max_features=100,  norm='l2', ngram_range=(1, 3)), 'Description'),
     remainder=StandardScaler())

X = column_trans.fit_transform(dataframe.drop(['target'], axis=1))

y = dataframe.target

In [15]:
y.nunique()

2

In [16]:
X.shape

(11537, 302)

In [17]:
input_dim = X.shape[1]

In [18]:
input_dim

302

In [19]:
df = pd.DataFrame.sparse.from_spmatrix(X)

df.columns = ['col_{}'.format(i) for i in range(input_dim)]

df['target']= y

df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  """


,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,...,col_293,col_294,col_295,col_296,col_297,col_298,col_299,col_300,col_301,target
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,-0.452479,0.950288,-0.829762,1
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,-0.555981,-0.299388,-0.511871,1
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,-0.555981,-0.299388,1.077583,1
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.226480,0.0,-0.400729,1.575125,1.395473,1
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.198113,0.0,0.133816,0.181835,0.0,-0.555981,-0.299388,-0.193980,1


## Train with Spark NLP Generic Classifier

**Building a pipeline**

The FeaturesAssembler is used to collect features from different columns. It can collect features from single value columns (anything which can be cast to a float, if casts fails then the value is set to 0), array columns or SparkNLP annotations (if the annotation is an embedding, it takes the embedding, otherwise tries to cast the 'result' field). The output of the transformer is a FEATURE_VECTOR annotation (the numeric vector is in the 'embeddings' field).

The GenericClassifierApproach takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations. The operation of the classifier is controled by the following methods:

*setEpochsNumber(int)* - Determines how many epochs the model is trained.

*setBatchSize(int)* - Sets the batch size during training.

*setLearningRate(float)* - Sets the learning rate.

*setValidationSplit(float)* - Sets the proportion of examples in the training set used for validation.

*setModelFile(string)* - Loads a model from the specified location and uses it instead of the default model.

*setFixImbalance(boolean)* - If set to true, it tries to balance the training set by weighting the classes according to the inverse of the examples they have.

*setFeatureScaling(string)* - Normalizes the feature factors using the specified method ("zscore", "minmax" or empty for no normalization).

*setOutputLogsPath(string)* - Sets the path to a folder where logs of training progress will be saved. No logs are generated if no path is specified.

In [20]:
spark_df = spark.createDataFrame(df)
spark_df.select(spark_df.columns[-10:]).show(2)

+-------+-------+-------+-------+-------+-------+-------------------+--------------------+-------------------+------+
|col_293|col_294|col_295|col_296|col_297|col_298|            col_299|             col_300|            col_301|target|
+-------+-------+-------+-------+-------+-------+-------------------+--------------------+-------------------+------+
|    0.0|    0.0|    0.0|    0.0|    0.0|    0.0|-0.4524794726808656|  0.9502875792756131|-0.8297616989552165|     1|
|    0.0|    0.0|    0.0|    0.0|    0.0|    0.0| -0.555981017719065|-0.29938816657135553|-0.5118709929431844|     1|
+-------+-------+-------+-------+-------+-------+-------------------+--------------------+-------------------+------+
only showing top 2 rows



In [21]:
from pyspark.sql.functions import col

spark_df.groupBy("target") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+------+-----+
|target|count|
+------+-----+
|     1| 8457|
|     0| 3080|
+------+-----+



In [22]:
(training_data, test_data) = spark_df.randomSplit([0.8, 0.2], seed = 100)

print("Training Dataset Count: " + str(training_data.count()))
print("Test Dataset Count: " + str(test_data.count()))

Training Dataset Count: 9234
Test Dataset Count: 2303


## Create a custom DL architecture with TF

In [ ]:
!pip install -q tensorflow==2.7.0 tensorflow_addons

In [24]:
from sparknlp_jsl.training import tf_graph

!mkdir gc_graph

tf_graph.print_model_params("generic_classifier")

generic_classifier parameters.
Parameter            Required   Default value        Description
hidden_layers        no         [200]                List of integers indicating the size of each hidden layer. For example: [100, 200, 300].
input_dim            yes        -                    Input dimension.
output_dim           yes        -                    Output dimension.
hidden_act           no         relu                 Activation function of hidden layers: relu, sigmoid, tanh or linear.
hidden_act_l2        no         0                    L2 regularization of hidden layer activations. Boolean (0 or 1).
hidden_weights_l2    no         0                    L2 regularization of hidden layer weights. Boolean (0 or 1).
batch_norm           no         0                    Batch normalization. Boolean (0 or 1).


In [25]:
DL_params = {"input_dim": input_dim, 
             "output_dim": y.nunique(), 
             "hidden_layers": [300, 200, 100], 
             "hidden_act": "tanh",
             'hidden_act_l2':1,
             'batch_norm':1}


tf_graph.build("generic_classifier",
               build_params=DL_params, 
               model_location="/content/gc_graph",
               model_filename="auto")

Instructions for updating:
Colocations handled automatically by placer.
generic_classifier graph exported to /content/gc_graph/gcl.302.2.pb


In [26]:
#or just use the one we already have in the repo

!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/generic_classifier_graph/pet.in1202D.out2.pb -P /content/gc_graph

In [27]:
from sparknlp_jsl.base import *

!mkdir logs

features_asm = FeaturesAssembler()\
    .setInputCols(['col_{}'.format(i) for i in range(X.shape[1])])\
    .setOutputCol("features")
      
gen_clf = GenericClassifierApproach()\
    .setLabelColumn("target")\
    .setInputCols(["features"])\
    .setOutputCol("prediction")\
    .setModelFile('/content/gc_graph/pet.in1202D.out2.pb')\
    .setEpochsNumber(50)\
    .setBatchSize(100)\
    .setFeatureScaling("zscore")\
    .setFixImbalance(True)\
    .setLearningRate(0.001)\
    .setOutputLogsPath("logs")\
    .setValidationSplit(0.2) # keep 20% of the data for validation purposes

clf_Pipeline = Pipeline(stages=[
    features_asm, 
    gen_clf])


In [28]:
%%time

# train 50 epochs (takes around 1 min)

clf_model = clf_Pipeline.fit(training_data)

CPU times: user 461 ms, sys: 72.2 ms, total: 533 ms
Wall time: 1min 10s


In [29]:
log_file_name = os.listdir("logs")[0]

with open("logs/"+log_file_name, "r") as log_file :
    print(log_file.read())

Training 50 epochs
Epoch 1/50	2.55s	Loss: 18.401125	ACC: 0.6340431	Validation ACC: 0.6513265
Epoch 2/50	1.20s	Loss: 15.552597	ACC: 0.7156368	Validation ACC: 0.67677313
Epoch 3/50	1.06s	Loss: 13.259397	ACC: 0.7665829	Validation ACC: 0.69680566
Epoch 4/50	0.70s	Loss: 10.283122	ACC: 0.8254132	Validation ACC: 0.7173795
Epoch 5/50	0.73s	Loss: 8.0004635	ACC: 0.8777306	Validation ACC: 0.68327016
Epoch 6/50	0.73s	Loss: 5.973315	ACC: 0.91341245	Validation ACC: 0.7298322
Epoch 7/50	0.71s	Loss: 4.4738674	ACC: 0.93827736	Validation ACC: 0.6876015
Epoch 8/50	0.66s	Loss: 3.3054392	ACC: 0.9539933	Validation ACC: 0.73741204
Epoch 9/50	0.68s	Loss: 2.1104422	ACC: 0.97482467	Validation ACC: 0.7330807
Epoch 10/50	0.65s	Loss: 1.5924187	ACC: 0.9847299	Validation ACC: 0.7504061
Epoch 11/50	0.68s	Loss: 1.3218652	ACC: 0.9833178	Validation ACC: 0.7471576
Epoch 12/50	0.66s	Loss: 1.6879158	ACC: 0.978338	Validation ACC: 0.7292907
Epoch 13/50	0.64s	Loss: 2.246441	ACC: 0.9761152	Validation ACC: 0.73253924
Epoch 14/5

In [30]:
pred_df = clf_model.transform(test_data)

In [31]:
pred_df.select('target','prediction.result').show()

+------+------+
|target|result|
+------+------+
|     1|   [1]|
|     1|   [1]|
|     0|   [1]|
|     1|   [0]|
|     0|   [0]|
|     1|   [1]|
|     0|   [1]|
|     1|   [1]|
|     1|   [1]|
|     1|   [1]|
|     0|   [1]|
|     1|   [0]|
|     1|   [1]|
|     1|   [1]|
|     1|   [1]|
|     1|   [1]|
|     1|   [1]|
|     1|   [0]|
|     1|   [1]|
|     1|   [1]|
+------+------+
only showing top 20 rows



In [32]:
preds_df = pred_df.select('target','prediction.result').toPandas()

# Let's explode the array and get the item(s) inside of result column out
preds_df['result'] = preds_df['result'].apply(lambda x : int(x[0]))


In [33]:
preds_df

,target,result
0,1,1
1,1,1
2,0,1
3,1,0
4,0,0
...,...,...
2298,1,1
2299,1,0
2300,1,1
2301,1,1


In [34]:
# We are going to use sklearn to evalute the results on test dataset
from sklearn.metrics import classification_report, accuracy_score

print (classification_report(preds_df['result'], preds_df['target'], digits=4))

print (accuracy_score(preds_df['result'], preds_df['target']))


              precision    recall  f1-score   support

           0     0.4430    0.4786    0.4601       585
           1     0.8175    0.7951    0.8061      1718

    accuracy                         0.7147      2303
   macro avg     0.6303    0.6369    0.6331      2303
weighted avg     0.7224    0.7147    0.7183      2303

0.7147199305254016


## get prediction for random input

In [35]:
pd.DataFrame([dataframe.loc[5191].to_dict()])

,Type,Age,Breed1,Gender,Color1,Color2,MaturitySize,FurLength,Vaccinated,Sterilized,Health,Fee,Description,PhotoAmt,target
0,Dog,2,Mixed Breed,Female,Black,Brown,Medium,Medium,Yes,Yes,Healthy,10,Friendly and pampered female puppy looking for...,2,1


In [36]:
input_X = column_trans.transform(pd.DataFrame([dataframe.loc[0].to_dict()]).drop(['target'], axis=1))

input_y = dataframe.target[0]


In [37]:
input_df = pd.DataFrame.sparse.from_spmatrix(input_X)

input_df.columns = ['col_{}'.format(i) for i in range(input_X.shape[1])]

input_df['target']= input_y

input_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  """


,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,...,col_293,col_294,col_295,col_296,col_297,col_298,col_299,col_300,col_301,target
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.452479,0.950288,-0.829762,1


In [38]:
input_spark_df = spark.createDataFrame(input_df)
input_spark_df.show(2)

+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-----

In [39]:
clf_model.transform(input_spark_df).select('target','prediction.result').show()

+------+------+
|target|result|
+------+------+
|     1|   [1]|
+------+------+



# Case Study: Alexa Review Classification

In [40]:
! wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Public/data/amazon_alexa.tsv

In [41]:
import pandas as pd
df = pd.read_csv('amazon_alexa.tsv', sep='\t')
df

,rating,date,variation,verified_reviews,feedback
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1
1,5,31-Jul-18,Charcoal Fabric,Loved it!,1
2,4,31-Jul-18,Walnut Finish,"Sometimes while playing a game, you can answer...",1
3,5,31-Jul-18,Charcoal Fabric,I have had a lot of fun with this thing. My 4 ...,1
4,5,31-Jul-18,Charcoal Fabric,Music,1
...,...,...,...,...,...
3145,5,30-Jul-18,Black Dot,"Perfect for kids, adults and everyone in betwe...",1
3146,5,30-Jul-18,Black Dot,"Listening to music, searching locations, check...",1
3147,5,30-Jul-18,Black Dot,"I do love these things, i have them running my...",1
3148,5,30-Jul-18,White Dot,Only complaint I have is that the sound qualit...,1


In [42]:
df.verified_reviews = df.verified_reviews.str.lower()

In [43]:
df.feedback.value_counts()

1    2893
0     257
Name: feedback, dtype: int64

In [44]:
from sklearn.compose import make_column_transformer
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


column_trans = make_column_transformer((OneHotEncoder(), ['rating','variation']),
     (TfidfVectorizer(max_features=1000,  norm='l2', ngram_range=(1, 3)), 'verified_reviews'))

X = column_trans.fit_transform(df.drop(['feedback'], axis=1))

y = df.feedback

In [45]:
sdf = pd.DataFrame.sparse.from_spmatrix(X)

sdf.columns = ['col_{}'.format(i) for i in range(X.shape[1])]

sdf['feedback']= y

sdf.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  """


,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,...,col_1012,col_1013,col_1014,col_1015,col_1016,col_1017,col_1018,col_1019,col_1020,feedback
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.324472,0.0,0.149662,0.0,0.0,0.0,0.0,0.0,1
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1


In [46]:
input_spark_df = spark.createDataFrame(sdf)

In [47]:
input_spark_df.show(5)

+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+-------------------+-------------------+-------------------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+-------------------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+-------------------+------+------+------+------+------+------+------+------+------+------+-------------------+------+------+------+------+------+------+------+------+------+------+------------------+------------------+------+-------+-------+-------+-------+-------+-------+-------------------+-------+-------+-------+-------+-------+-------+-------------------+-------+-------+-------+-------+-------+-------+-------+-------+-------

In [48]:
(training_data, test_data) = input_spark_df.randomSplit([0.8, 0.2], seed = 100)

print("Training Dataset Count: " + str(training_data.count()))
print("Test Dataset Count: " + str(test_data.count()))

Training Dataset Count: 2529
Test Dataset Count: 621


In [49]:
from sparknlp_jsl.base import *

features_asm = FeaturesAssembler()\
    .setInputCols(['col_{}'.format(i) for i in range(X.shape[1])])\
    .setOutputCol("features")
        
gen_clf = GenericClassifierApproach()\
    .setLabelColumn("feedback")\
    .setInputCols(["features"])\
    .setOutputCol("prediction")\
    .setModelFile('/content/gc_graph/pet.in1202D.out2.pb')\
    .setEpochsNumber(50)\
    .setBatchSize(100)\
    .setFeatureScaling("zscore")\
    .setFixImbalance(True)\
    .setLearningRate(0.001)\
    .setOutputLogsPath("logs")\

clf_Pipeline = Pipeline(stages=[
    features_asm, 
    gen_clf])

clf_model = clf_Pipeline.fit(training_data)


In [50]:
pred_df = clf_model.transform(test_data)

preds_df = pred_df.select('feedback','prediction.result').toPandas()

# Let's explode the array and get the item(s) inside of result column out
preds_df['result'] = preds_df['result'].apply(lambda x : int(x[0]))

# We are going to use sklearn to evalute the results on test dataset
from sklearn.metrics import classification_report, accuracy_score

print(classification_report(preds_df['result'], preds_df['feedback'], digits=4))

print(accuracy_score(preds_df['result'], preds_df['feedback']))


              precision    recall  f1-score   support

           0     0.7679    0.9556    0.8515        45
           1     0.9965    0.9774    0.9869       576

    accuracy                         0.9758       621
   macro avg     0.8822    0.9665    0.9192       621
weighted avg     0.9799    0.9758    0.9770       621

0.9758454106280193
